In [9]:
### Hayden Gallo
### Bucci Lab
### 6/17/25
### Gapfilling genome scale metabolic models for MDSINE FBA

import numpy as np
#from dfba import DfbaModel, ExchangeFlux, KineticVariable
import cobra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import sys
import os
import gurobipy
import gzip
from cobra.io import read_sbml_model
from cobra.flux_analysis.gapfilling import GapFiller
from pathlib import Path
from helper_functions import *

ModuleNotFoundError: No module named 'helper_functions'

In [2]:
### Load in the universal bacterial GEM from carveme 

with gzip.open("/Users/haydengallo/anaconda3/envs/carveme/lib/python3.13/site-packages/carveme/data/generated/universe_bacteria.xml.gz", "rt") as f:
    universal = read_sbml_model(f)




Set parameter Username
Academic license - for non-commercial use only - expires 2026-05-30


In [3]:
### Load in the AGORA model I want to gapfill

### load the cobra models into memory i guess

cobra_models_dir = Path('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/processed_data_filtered_RC_all_cohorts/all_strains_agora')

# Make the data and validation Study objects
cobra_models = sorted(cobra_models_dir.glob('*.mat'))
cobra_models = {f.stem : f for f in cobra_models}


### Just loading the models needed in 

loaded_models = {}

#count = 0

for key in cobra_models:
    #if count == 1:
    #    break
    #print(key.split('_'))
    model_name = key.split('_')[0] + '_' + key.split('_')[1]
    print(model_name)
    model = cobra.io.load_matlab_model(cobra_models[key])
    loaded_models[model_name] = model
    #count+=1


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


Allobaculum_stercoricanis


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


Bifidobacterium_pseudolongum


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


...

Staphylococcus_xylosus


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


Turicibacter_sp


No defined compartments in model relaxedModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


n_massi


In [4]:
### Load in the Media conditions to gapfill from 

### Load in diet data that will be applied in intervals 

RC_diet = pd.read_csv('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/RC_diet_add.csv', header=None)
RC_diet.columns = ['reaction','fluxValue','upper_bound']
RC_diet = RC_diet[['reaction', 'fluxValue']]
RC_diet['fluxValue'] =  -1.0*(np.double(RC_diet['fluxValue']))

## Multiply by 5g over 12 hours
#RC_diet['fluxValue'] = ((5/48)*RC_diet['fluxValue'])
RC_diet.head()
RC_diet_dict = dict(zip(RC_diet['reaction'], RC_diet['fluxValue']))
RC_diet_dict

{'EX_12kltchca(e)': 0.0,
 'EX_drib(e)': 0.0,
 'EX_5oxpro(e)': 0.131,
 'EX_ala_l(e)': 0.836,
 'EX_all_d(e)': 0.029,
 'EX_arab_l(e)': 0.055,
 'EX_asn_l(e)': 0.0,
 'EX_asp_l(e)': 0.398,
 'EX_cellb(e)': 0.0,
 'EX_cholate(e)': 0.0,
 'EX_cit(e)': 1.046,
 'EX_cys_l(e)': 1.989,
 'EX_dca(e)': 0.046,
 'EX_dchac(e)': 0.028,
 'EX_ddca(e)': 0.038,
 'EX_galt(e)': 0.0,
 'EX_fru(e)': 2.506,
 'EX_fuc_l(e)': 0.0,
 'EX_fum(e)': 0.072,
 'EX_gal(e)': 0.048,
 'EX_glc_d(e)': 0.612,
 'EX_glu_l(e)': 0.017,
 'EX_gly(e)': 0.072,
 'EX_hxa(e)': 0.059,
 'EX_his_l(e)': 0.0,
 'EX_indole(e)': 0.0,
 'EX_ile_l(e)': 0.387,
 'EX_leu_l(e)': 0.644,
 'EX_lnlcacp(e)': 5.732,
 'EX_hc02191(e)': 0.0,
 'EX_lys_l(e)': 0.0,
 'EX_mal_l(e)': 0.382,
 'EX_melib(e)': 0.021,
 'EX_met_l(e)': 4.781,
 'EX_ttdca(e)': 0.696,
 'EX_acgam(e)': 0.0,
 'EX_nac(e)': 0.172,
 'EX_octa(e)': 0.13,
 'EX_orn_l(e)': 0.0,
 'EX_hdca(e)': 0.599,
 'EX_phe_l(e)': 0.115,
 'EX_c10164(e)': 0.064,
 'EX_pro_l(e)': 2.307,
 'EX_pyr(e)': 0.032,
 'EX_raffin(e)': 1.192,


In [8]:
### Set the medium for the model of interest

loaded_models['Allobaculum_stercoricanis'].medium = RC_diet_dict



KeyError: 'EX_12kltchca(e)'

In [ ]:
# Assume `model` is your AGORA model and `model.medium` is set
gapfiller = GapFiller(model, universal, demand_reactions=False)
solutions = gapfiller.fill()

# Add first solution to your model
for rxn in solutions[0]:
    model.add_reactions([rxn])
